In [1]:
import numpy as np
from machine_learning.xgb_def import XGBOptimizer
from machine_learning.reglog_def import REGLOGOptimizer
import pandas as pd
from opt_funct.data_fuct import functions
from sklearn.model_selection import train_test_split
from machine_learning.SVM_defopt import SVMOptimizer
from itertools import combinations
from sklearn.preprocessing import StandardScaler
from opt_funct.data_fuct import STRATEGY_XGBOOST_BUY
from opt_funct.data_fuct import STRATEGY_REGLOG_BUY
from opt_funct.data_fuct import STRATEGY_SVC_BUY
from opt_funct.data_fuct import STRATEGY_XGBOOST_REGLOG_BUY
from opt_funct.data_fuct import STRATEGY_XGBOOST_SVC_BUY
from opt_funct.data_fuct import STRATEGY_REGLOG_SVC_BUY
from opt_funct.data_fuct import STRATEGY_XGBOOST_REGLOG_SVC_BUY
from opt_funct.data_fuct import STRATEGY_XGBOOST_SELL
from opt_funct.data_fuct import STRATEGY_REGLOG_SELL
from opt_funct.data_fuct import STRATEGY_SVC_SELL
from opt_funct.data_fuct import STRATEGY_XGBOOST_REGLOG_SELL
from opt_funct.data_fuct import STRATEGY_XGBOOST_SVC_SELL
from opt_funct.data_fuct import STRATEGY_REGLOG_SVC_SELL
from opt_funct.data_fuct import STRATEGY_XGBOOST_REGLOG_SVC_SELL
import optuna
from opt_funct.data_fuct import Operation



/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = functions.clean_ds(pd.read_csv('./data/aapl_5m_train.csv'))
df

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,X_t-1,X_t-2,X_t-3,Pt_5,RSI,Y_BUY,Y_SELL
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0,NaN,NaN,NaN,131.448898,NaN,False,True
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0,132.809997,NaN,NaN,131.119995,NaN,False,True
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0,131.889999,132.809997,NaN,130.875000,NaN,False,True
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0,132.059997,131.889999,132.809997,130.509902,NaN,False,True
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0,132.250000,132.059997,131.889999,130.889999,NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39574,19788,1672432800,0,2022-12-30 20:40:00,129.029998,129.130004,128.919998,129.054992,833767.0,129.039993,129.020004,128.910003,NaN,56.509324,False,False
39575,19789,1672433100,0,2022-12-30 20:45:00,129.059997,129.399993,129.020004,129.380004,2136910.0,129.054992,129.039993,129.020004,NaN,60.700323,False,False
39576,19790,1672433400,0,2022-12-30 20:50:00,129.375000,129.929992,129.330001,129.764404,2879267.0,129.380004,129.054992,129.039993,NaN,64.854367,False,False
39577,19791,1672433700,0,2022-12-30 20:55:00,129.764999,129.949996,129.619995,129.949996,3896812.0,129.764404,129.380004,129.054992,NaN,66.620919,False,False


**Entrenamos nuestros modelos para compra y venta**

In [3]:
# Buy
close_data = df[['Timestamp','Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_BUY']].dropna()
x = close_data.drop(columns=['Y_BUY'])
y = close_data[['Y_BUY']]
x_train_buy, x_test_buy, y_train_buy, y_test_buy = train_test_split(x, y, test_size=0.3, shuffle=False)

close_data2 = df[['Timestamp','Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_SELL']].dropna()
x = close_data2.drop(columns=['Y_SELL'])
y = close_data2[['Y_SELL']]
x_train_sell, x_test_sell, y_train_sell, y_test_sell = train_test_split(x, y, test_size=0.3, shuffle=False)

In [4]:
# Escalado para el conjunto de compra
scaler_buy = StandardScaler()
x_train_buy_scaled = scaler_buy.fit_transform(x_train_buy)
x_test_buy_scaled = scaler_buy.transform(x_test_buy)

# Escalado para el conjunto de venta
scaler_sell = StandardScaler()
x_train_sell_scaled = scaler_sell.fit_transform(x_train_sell)
x_test_sell_scaled = scaler_sell.transform(x_test_sell)

In [5]:
# buy
xgboost_optimizer = XGBOptimizer(x_train_buy, y_train_buy, x_test_buy, y_test_buy)
xgboost_best_params_buy, best_model_xgboost_buy = xgboost_optimizer.xgb_optuna()

reg_log_optimizer = REGLOGOptimizer(x_train_buy, y_train_buy, x_test_buy, y_test_buy)
reg_log_best_params_buy, best_model_reg_log_buy = reg_log_optimizer.reglog_optuna()

SVC_optimizer = SVMOptimizer(x_train_buy_scaled, y_train_buy, x_test_buy_scaled, y_test_buy)
SVC_best_params_buy, best_model_SVC_buy = SVC_optimizer.svm_optuna()

# sell
xgboost_optimizer = XGBOptimizer(x_train_sell, y_train_sell, x_test_sell, y_test_sell)
xgboost_best_params_sell, best_model_xgboost_sell = xgboost_optimizer.xgb_optuna()

reg_log_optimizer = REGLOGOptimizer(x_train_sell, y_train_sell, x_test_sell, y_test_sell)
reg_log_best_params_sell, best_model_reg_log_sell = reg_log_optimizer.reglog_optuna()

SVC_optimizer = SVMOptimizer(x_train_sell_scaled, y_train_sell, x_test_sell_scaled, y_test_sell)
SVC_best_params_sell, best_model_SVC_sell = SVC_optimizer.svm_optuna()

[I 2024-03-10 14:20:39,297] A new study created in memory with name: no-name-cd39bf7c-4725-42b2-979b-d1800924920c
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [14:20:41] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "gamma", "max_depth", "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-03-10 14:20:41,145] Trial 0 finished with value: 0.6788796703602651 and parameters: {'n_estimators': 79, 'max_depth': 82, 'max_leaves': 48, 'learning_rate': 1.3048935930093684, 'booster': 'gblinear', 'gamma': 44.7240098687193, 'reg_alpha': 3.4564780065691787, 'reg_lambda': 1.9225311963027785}. Best is trial 0 with value: 0.6788796703602651.
[I 2024-03-10 14:20:41,185] Trial 1 finished with value: 0.0 and parameters: {'n_estimators': 29, 'max_depth': 52, 'max_leaves': 31, 'learning_rate': 0.30021663978251595, 'booster': 'gbtree', 'gamma': 19.938501388548183, 'reg_alpha': 5.317923084709651, 'reg_lambda': 5.339532964188512}

F1: 0.6788796703602651
Best hyperparameters: {'n_estimators': 79, 'max_depth': 82, 'max_leaves': 48, 'learning_rate': 1.3048935930093684, 'booster': 'gblinear', 'gamma': 44.7240098687193, 'reg_alpha': 3.4564780065691787, 'reg_lambda': 1.9225311963027785}
Execution time: 0.04293568134307861 minutes


[I 2024-03-10 14:21:00,203] Trial 0 finished with value: 0.6788796703602651 and parameters: {'C': 8.415114976536797, 'fit_intercept': True, 'l1_ratio': 0.5961195432058555}. Best is trial 0 with value: 0.6788796703602651.
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-10 14:21:18,137] Trial 1 finished with value: 0.6788796703602651 and parameters: {'C': 9.176761007780557, 'fit_intercept': True, 'l1_ratio': 0.5110268772117587}. Best is trial 0 with value: 0.6788796703602651.
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(

F1 Score: 0.6788796703602651
Best hyperparameters: {'C': 8.415114976536797, 'fit_intercept': True, 'l1_ratio': 0.5961195432058555}
Execution time: 2.9227911472320556 minutes


/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-10 14:23:50,683] Trial 0 finished with value: 0.6788796703602651 and parameters: {'C': 8.484868143281304, 'kernel': 'linear', 'degree': 2, 'gamma': 'auto'}. Best is trial 0 with value: 0.6788796703602651.
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-10 14:24:06,234] Trial 1 finished with value: 0.2694350555287301 and parameters: {'C': 3.982339307541823, 'kernel': 'rbf', 'degree': 3, 'gamma': 'scale'}. Best is trial 0 with value: 0.6788796703602651.
/home/rodo/.pyenv/versions/3.11.8/lib/pytho

...

F1: 0.5987625573814117
Best hyperparameters: {'n_estimators': 50, 'max_depth': 72, 'max_leaves': 24, 'learning_rate': 0.07697139485549398, 'booster': 'gbtree', 'gamma': 9.791785698092811, 'reg_alpha': 7.073444753043538, 'reg_lambda': 3.4338594092038464}
Execution time: 0.1406004548072815 minutes


[I 2024-03-10 14:26:16,036] Trial 1 finished with value: 0.0 and parameters: {'C': 1.8404539979141032, 'fit_intercept': False, 'l1_ratio': 0.694955846793955}. Best is trial 0 with value: 0.0.
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-10 14:26:16,181] Trial 2 finished with value: 0.0 and parameters: {'C': 4.668299979764073, 'fit_intercept': True, 'l1_ratio': 0.49803303601363047}. Best is trial 0 with value: 0.0.
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-10 14:26:16,328] Trial 3 finished

F1 Score: 0.0
Best hyperparameters: {'C': 6.470582711477581, 'fit_intercept': False, 'l1_ratio': 0.9180608261618894}
Execution time: 0.023079137007395428 minutes


[I 2024-03-10 14:26:33,175] Trial 0 finished with value: 0.5899300129102398 and parameters: {'C': 7.071487000708991, 'kernel': 'rbf', 'degree': 4, 'gamma': 'scale'}. Best is trial 0 with value: 0.5899300129102398.
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-10 14:26:45,972] Trial 1 finished with value: 0.0 and parameters: {'C': 3.2188086380700183, 'kernel': 'poly', 'degree': 3, 'gamma': 'scale'}. Best is trial 0 with value: 0.5899300129102398.
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-pack

F1 Score: 0.6510887486497242
Best hyperparameters: {'C': 9.114976982321487, 'kernel': 'linear', 'degree': 3, 'gamma': 'scale'}
Execution time: 2.252304259936015 minutes


In [6]:
# Indicadores Buy
close_data['xg_boost_buy'] = best_model_xgboost_buy.predict(x)
close_data['reg_log_buy'] = best_model_reg_log_buy.predict(x)
close_data['SVC_buy'] = best_model_SVC_buy.predict(x)
# Indicadores Sell
close_data = close_data.drop(columns=['Y_BUY','RSI','X_t-1','X_t-2','X_t-3'])
close_data['xg_boost_sell'] = best_model_xgboost_sell.predict(x)
close_data['reg_log_sell'] = best_model_reg_log_sell.predict(x)
close_data['SVC_sell'] = best_model_SVC_sell.predict(x)
# Convertimos a boleanos
close_data['xg_boost_buy'] = np.where(close_data['xg_boost_buy']==1, True, False)
close_data['xg_boost_sell'] = np.where(close_data['xg_boost_sell']==1, True, False)
close_data

/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


,Timestamp,Close,xg_boost_buy,reg_log_buy,SVC_buy,xg_boost_sell,reg_log_sell,SVC_sell
27,1609778700,129.413299,True,True,True,False,False,False
28,1609779000,129.148498,True,True,True,False,False,False
29,1609779300,128.820007,True,True,True,False,False,False
30,1609779600,128.580001,True,True,True,False,False,False
31,1609779900,128.351699,True,True,True,False,False,False
...,...,...,...,...,...,...,...,...
39574,1672432800,129.054992,True,True,True,False,False,False
39575,1672433100,129.380004,True,True,True,False,False,False
39576,1672433400,129.764404,True,True,True,False,False,False
39577,1672433700,129.949996,True,True,True,False,False,False


**Mostramos nuestras combinaciones posibles de compra y venta**

In [7]:
indicadores_compra = ['xg_boost_buy', 'reg_log_buy' , 'SVC_buy']
indicadores_venta = ['xg_boost_sell', 'reg_log_sell' , 'SVC_sell']

combinaciones_indicadores_compra = []
combinaciones_indicadores_venta = []

for i in range(1, len(indicadores_compra) + 1):
    combinaciones_indicadores_compra.extend(combinations(indicadores_compra, i))

for i in range(1, len(indicadores_venta) + 1):
    combinaciones_indicadores_venta.extend(combinations(indicadores_venta, i))
    
combinaciones_indicadores_compra, combinaciones_indicadores_venta

([('xg_boost_buy',),
  ('reg_log_buy',),
  ('SVC_buy',),
  ('xg_boost_buy', 'reg_log_buy'),
  ('xg_boost_buy', 'SVC_buy'),
  ('reg_log_buy', 'SVC_buy'),
  ('xg_boost_buy', 'reg_log_buy', 'SVC_buy')],
 [('xg_boost_sell',),
  ('reg_log_sell',),
  ('SVC_sell',),
  ('xg_boost_sell', 'reg_log_sell'),
  ('xg_boost_sell', 'SVC_sell'),
  ('reg_log_sell', 'SVC_sell'),
  ('xg_boost_sell', 'reg_log_sell', 'SVC_sell')])

In [8]:
cash = 1_000_000
active_operations = []
com = 0.00125  # comision en GBM
strategy_value = [1_000_000]


Calculamos la mejor combinacion para compra

In [9]:
best_global_strategy = {'name': None, 'value': float('-inf')}

def optimize(trial):
    # Definición de los parámetros a optimizar
    stop_loss_long = trial.suggest_float('stop_loss_long', 0.01, 0.3)
    take_profit_long = trial.suggest_float('take_profit_long', 0.01, 0.3)
    n_shares = trial.suggest_int('n_shares', 10, 100)
    
    # Inicialización de la estrategia con los parámetros actuales
    xg_boost_buy = STRATEGY_XGBOOST_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    reglog_buy = STRATEGY_REGLOG_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    svc_buy = STRATEGY_SVC_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_reglog_buy = STRATEGY_XGBOOST_REGLOG_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_svc_buy = STRATEGY_XGBOOST_SVC_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    reglog_svc_buy = STRATEGY_REGLOG_SVC_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_reglog_svc_buy = STRATEGY_XGBOOST_REGLOG_SVC_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    strategy_values = {
        'xgboost_buy': xg_boost_buy.run_strategy(),
        'reglog_buy': reglog_buy.run_strategy(),
        'svc_buy': svc_buy.run_strategy(),
        'xgboost_reglog_buy': xgboost_reglog_buy.run_strategy(),
        'xgboost_svc_buy': xgboost_svc_buy.run_strategy(),
        'reglog_svc_buy': reglog_svc_buy.run_strategy(),
        'xgboost_reglog_svc_buy': xgboost_reglog_svc_buy.run_strategy()
    }

    # Encuentra el mayor valor de estrategia y su correspondiente estrategia
    best_strategy_name = max(strategy_values, key=strategy_values.get)
    best_strategy_value = strategy_values[best_strategy_name]

    if best_strategy_value > best_global_strategy['value']:
        best_global_strategy['name'] = best_strategy_name
        best_global_strategy['value'] = best_strategy_value

    
    # Retorna el valor de la mejor estrategia
    return best_strategy_value

In [11]:
# Inicializar y ejecutar la optimización
study = optuna.create_study(direction='maximize')
study.optimize(optimize, n_trials=5, n_jobs=-1)

# Los mejores parámetros encontrados en el mejor trial
best_params = study.best_trial.params
best_value = study.best_trial.value

# Comparar con el mejor valor global previamente encontrado y el nombre de la estrategia
best_strategy_name = best_global_strategy['name']
best_strategy_value = best_global_strategy['value']

# Imprimir los resultados, incluido el nombre de la mejor estrategia global y su valor
print(f"Best buy overall strategy: {best_strategy_name} with value: {best_strategy_value}")
print("Best buy strategy parameters:", best_params)

[I 2024-03-10 14:29:57,067] A new study created in memory with name: no-name-846019bc-ff1f-40ca-94a7-2d32d555e31c
[I 2024-03-10 14:44:50,881] Trial 3 finished with value: 995630.7023608788 and parameters: {'stop_loss_long': 0.21450411733572566, 'take_profit_long': 0.14880109689733623, 'n_shares': 64}. Best is trial 3 with value: 995630.7023608788.
[I 2024-03-10 14:51:03,088] Trial 4 finished with value: 985519.7594078361 and parameters: {'stop_loss_long': 0.23669202146131535, 'take_profit_long': 0.03827515911790308, 'n_shares': 41}. Best is trial 3 with value: 995630.7023608788.
[I 2024-03-10 14:51:03,833] Trial 0 finished with value: 994875.6781827752 and parameters: {'stop_loss_long': 0.029367668813473177, 'take_profit_long': 0.209856180082254, 'n_shares': 42}. Best is trial 3 with value: 995630.7023608788.
[I 2024-03-10 15:05:12,283] Trial 2 finished with value: 980076.1293588501 and parameters: {'stop_loss_long': 0.22669637642235968, 'take_profit_long': 0.034417754596144746, 'n_sha

Best buy overall strategy: xgboost_buy with value: 1008368.972719736
Best buy strategy parameters: {'stop_loss_long': 0.13204837666998107, 'take_profit_long': 0.16924732670477438, 'n_shares': 14}


Calculamos la mejor combinacion para venta

In [12]:
best_global_sell_strategy = {'name': None, 'value': float('-inf')}

def optimize_sell(trial):
    # Definición de los parámetros a optimizar
    stop_loss_short = trial.suggest_float('stop_loss_short', 0.01, 0.3)
    take_profit_short = trial.suggest_float('take_profit_short', 0.01, 0.3)
    n_shares = trial.suggest_int('n_shares', 10, 100)
    
    # Inicialización de la estrategia con los parámetros actuales
    xg_boost_sell = STRATEGY_XGBOOST_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    reglog_sell = STRATEGY_REGLOG_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    svc_sell = STRATEGY_SVC_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_reglog_sell = STRATEGY_XGBOOST_REGLOG_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_svc_sell = STRATEGY_XGBOOST_SVC_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    reglog_svc_sell = STRATEGY_REGLOG_SVC_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_reglog_svc_sell = STRATEGY_XGBOOST_REGLOG_SVC_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    strategy_values = {
        'xgboost_sell': xg_boost_sell.run_strategy(),
        'reglog_sell': reglog_sell.run_strategy(),
        'svc_sell': svc_sell.run_strategy(),
        'xgboost_reglog_sell': xgboost_reglog_sell.run_strategy(),
        'xgboost_svc_sell': xgboost_svc_sell.run_strategy(),
        'reglog_svc_sell': reglog_svc_sell.run_strategy(),
        'xgboost_reglog_svc_sell': xgboost_reglog_svc_sell.run_strategy()
    }

    # Encuentra el mayor valor de estrategia y su correspondiente estrategia
    best_strategy_name = max(strategy_values, key=strategy_values.get)
    best_strategy_value = strategy_values[best_strategy_name]

    if best_strategy_value > best_global_sell_strategy['value']:
        best_global_sell_strategy['name'] = best_strategy_name
        best_global_sell_strategy['value'] = best_strategy_value

    
    # Retorna el valor de la mejor estrategia
    return best_strategy_value

In [13]:
# Inicializar y ejecutar la optimización
study = optuna.create_study(direction='maximize')
study.optimize(optimize_sell, n_trials=5, n_jobs=-1)

# Los mejores parámetros encontrados en el mejor trial
best_params = study.best_trial.params
best_value = study.best_trial.value

# Comparar con el mejor valor global previamente encontrado y el nombre de la estrategia
best_strategy_name = best_global_sell_strategy['name']
best_strategy_value = best_global_sell_strategy['value']

# Imprimir los resultados, incluido el nombre de la mejor estrategia global y su valor
print(f"Best sell overall strategy: {best_strategy_name} with value: {best_strategy_value}")
print("Best sell strategy parameters:", best_params)

[I 2024-03-10 15:07:08,720] A new study created in memory with name: no-name-f2b2563f-d426-47c8-ab18-faf13618103e
[I 2024-03-10 15:17:22,002] Trial 0 finished with value: 1000000.0 and parameters: {'stop_loss_short': 0.18492882267669516, 'take_profit_short': 0.2953830666480811, 'n_shares': 40}. Best is trial 0 with value: 1000000.0.
[I 2024-03-10 15:17:47,790] Trial 4 finished with value: 1000000.0 and parameters: {'stop_loss_short': 0.18913361634027392, 'take_profit_short': 0.16169458037084167, 'n_shares': 48}. Best is trial 0 with value: 1000000.0.
[I 2024-03-10 15:37:00,173] Trial 3 finished with value: 27957674.542927396 and parameters: {'stop_loss_short': 0.1500294682690866, 'take_profit_short': 0.2465303146580537, 'n_shares': 10}. Best is trial 3 with value: 27957674.542927396.
[I 2024-03-10 15:43:25,027] Trial 1 finished with value: 192071745.84274292 and parameters: {'stop_loss_short': 0.2481841083567563, 'take_profit_short': 0.18344160897836873, 'n_shares': 53}. Best is trial 

Best sell overall strategy: xgboost_sell with value: 192071745.84274292
Best sell strategy parameters: {'stop_loss_short': 0.2481841083567563, 'take_profit_short': 0.18344160897836873, 'n_shares': 53}
